# Praktikum Pengantar Pembelajaran Mesin


---
## Bab 4. Klasifikasi Naive Bayes

Nama : Arthurito Nehemia Gilbert Eduardo Luke Keintjem

NIM : 225150207111047

Kelas : PPM TIF-A

### 1) Import Data

Unduh dataset yang akan digunakan pada praktikum kali ini. Anda dapat menggunakan aplikasi wget untuk mendowload dataset dan menyimpannya dalam Google Colab. Jalankan cell di bawah ini untuk mengunduh dataset

In [2]:
! wget https://gist.githubusercontent.com/netj/8836201/raw/6f9306ad21398ea43cba4f7d537619d0e07d5ae3/iris.csv

--2024-03-28 04:26:28--  https://gist.githubusercontent.com/netj/8836201/raw/6f9306ad21398ea43cba4f7d537619d0e07d5ae3/iris.csv
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3975 (3.9K) [text/plain]
Saving to: ‘iris.csv’

iris.csv            100%[===================>]   3.88K  --.-KB/s    in 0s      

2024-03-28 04:26:28 (41.4 MB/s) - ‘iris.csv’ saved [3975/3975]



Setelah dataset berhasil diunduh, langkah berikutnya adalah membaca dataset dengan memanfaatkan fungsi **readcsv** dari library pandas. Lakukan pembacaan berkas csv ke dalam dataframe dengan nama **data** menggunakan fungsi **readcsv**. Jangan lupa untuk melakukan import library pandas terlebih dahulu


In [3]:
import pandas as pd
import numpy as np
data = pd.read_csv('iris.csv')



Cek isi dataset Anda dengan menggunakan perintah **head()**

In [4]:
data.head()

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa
3,4.6,3.1,1.5,0.2,Setosa
4,5.0,3.6,1.4,0.2,Setosa


## 2) Membagi data menjadi data latih dan data uji

Metode pembelajaran mesin memerlukan dua jenis data :


1.   Data latih : Digunakan untuk proses training metode klasifikasi
2.   Data uji : Digunakan untuk proses evaluasi metode klasifikasi

Data uji dan data latih perlu dibuat terpisah (mutualy exclusive) agar hasil evaluasi lebih akurat.

Data uji dan data latih dapat dibuat dengan cara membagi dataset dengan rasio tertentu, misalnya 80% data latih dan 20% data uji.

Library Scikit-learn memiliki fungsi [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) pada modul **model_selection** untuk membagi dataset menjadi data latih dan data uji. Bagilah dataset anda menjadi dua, yaitu **data_latih** dan **data_uji**.


In [5]:
from sklearn.model_selection import train_test_split
data_latih, data_uji = train_test_split(data, test_size = 0.2)

Tampilkan banyaknya data pada **data_latih** dan **data_uji**. Seharusnya **data_latih** terdiri dari 120 data, dan **data_uji** terdiri dari 30 data

In [6]:
print(data_uji.shape[0])
print(data_latih.shape[0])

30
120


Pisahkan label/kategori dari data uji menjadi variabel tersendiri. Beri nama variabelnya  **label_uji**

In [7]:
label_uji = data_uji.pop('variety')

In [8]:
data_uji.head()

,sepal.length,sepal.width,petal.length,petal.width
28,5.2,3.4,1.4,0.2
1,4.9,3.0,1.4,0.2
138,6.0,3.0,4.8,1.8
42,4.4,3.2,1.3,0.2
23,5.1,3.3,1.7,0.5


## 3) Menghitung Prior

Tahapan pertama pada algoritma Naive Bayes adalah perhitungan prior. Prior suatu kelas merupakan peluang awal munculnya kelas tersebut. Atau dengan kata lain, prior merupakan frekuensi relatif dari suatu kelas terhadap keseluruhan data.

Buatlah fungsi bernama **hitung_prior** yang berfungsi menghitung nilai prior dari masing-masing kelas yang terdapat pada data latih. Fungsi ini menerima input berupa list kelas dari data latih.

In [9]:
from collections import Counter
def hitung_prior(list_kelas):
  n_data = len(list_kelas)
  prior = Counter(list_kelas)
  for key in prior.keys():
    prior[key]=prior[key]/n_data
  return prior

Ujilah fungsi **hitung_prior** menggunakan kelas dari variabel **data_latih**. Amati hasilnya

In [10]:
prior = hitung_prior(data_latih['variety'])
print(prior)

# print(sum(prior.values()))

Counter({'Versicolor': 0.375, 'Virginica': 0.31666666666666665, 'Setosa': 0.30833333333333335})


## 4) Menghitung *likelihood*

Praktikum kali ini mengimplementasikan metode Naive Bayes menggunakan datset iris. Semua fitur pada dataset iris memiliki tipe numerik. Dengan demikian, perhitungan *likelihood* dilakukan menggunakan Gaussian Model.

Tahapan yang perlu dilakukan sebelum perhitungan *likelihood* adalah menghitung rata-rata dan deviasi standar masing-masing fitur per kelas. Buatlah fungsi **hitung_rata2_std_kelas** yang berfungsi menghitung rata-rata dan standar deviasi per fitur dan kelas

In [17]:
def hitung_rata2_std_kelas(input_data):
  list_columns = input_data.columns[:-1]
  class_column_name = input_data.columns[-1]
  list_class = set(input_data[class_column_name])
  rata2 = {}
  std = {}
  for column in list_columns:
    for a_class in list_class:
      rata2[(a_class,column)] = input_data.loc[input_data[class_column_name]==a_class][column].mean()
      std[(a_class,column)] = input_data.loc[input_data[class_column_name]==a_class][column].std()
  return(rata2,std)

Buatlah fungsi **hitung_likelihood_gaussian** yang bertujuan menghitung nilai likelihood suatu nilai terhadap rata-rata dan deviasi standar tertentu.

In [18]:
import math
def hitung_likelihood_gaussian(data,rata2,std):
  hasil = (1/math.sqrt(2*math.pi*(std**2)))*math.exp((-1*((data-rata2)**2))/(2*(std**2)))
  return hasil

## 5) Proses *training*
Proses training pada Naive Bayes dengan model Gaussian dilakukan untuk menghitung prior serta  rata-rata dan deviasi standar pada masing-msaing fitur dan kelas.

In [19]:
def training_naive_bayes_gaussian(data_latih):
  class_column_name = data_latih.columns[-1]
  prior = hitung_prior(data_latih[class_column_name])
  (rata2,std) = hitung_rata2_std_kelas(data_latih)
  list_class = set(data_latih[class_column_name])
  list_columns = data_latih.columns[:-1]
  model = {}
  model['prior'] = prior
  model['rata2'] = rata2
  model['std'] = std
  model['list_class'] = list_class
  model['list_columns'] = list_columns
  return model

Lakukanlah proses training dan simpan hasil training pada variabel **model**

In [20]:
# Sample training data (replace this with your actual training data)
# data_latih = pd.DataFrame({
#     'Feature1': [1, 2, 3, 4, 5],
#     'Feature2': [0.5, 1.5, 2.5, 3.5, 4.5],
#     'Class': ['A', 'A', 'B', 'B', 'B']
# })

# Training the model
model = training_naive_bayes_gaussian(data_latih)

# Printing the model
# print("Model:")
# print("a. Nilai prior:", model['prior'])
# print("b. Nilai rata-rata per fitur:", model['rata2'])
# print("c. Nilai standar deviasi per fitur:", model['std'])
# print("d. Daftar kelas pada data:", model['list_class'])
# print("e. Daftar kolom pada data:", model['list_columns'])

## 6) Proses *testing*
Proses testing dilakukan dengan menghitung nilai posterior dari data uji berdasarkan nilai model yang diperoleh saat training. Nilai posterior diperoleh dari perkalian prior dengan *likelihood* masing-masing fiturnya. Setelah itu, penentuan kelas data uji dilakukan berdasarkan nilai posterior terbesar

In [21]:
def testing_naive_bayes_gaussian(model,data_uji):
  prior = model['prior']
  rata2 = model['rata2']
  std = model['std']
  list_class = model['list_class']
  list_columns = model['list_columns']
  posterior = dict.fromkeys(list_class,1)
  for a_class in list_class:
    for column in list_columns:
      posterior[a_class] = posterior[a_class]*hitung_likelihood_gaussian(data_uji[column],rata2[(a_class,column)],std[(a_class,column)])
    posterior[a_class] = posterior[a_class]*prior[a_class]
  kelas_uji = max(posterior, key = posterior.get)
  return kelas_uji

Ujilah fungsi **testing_naive_bayes_gaussian** menggunakan data uji pertama. Bandingkan hasilnya dengan nilai label sebenarnya.

In [22]:
indeks_uji = 0
prediksi = testing_naive_bayes_gaussian(model,data_uji.iloc[indeks_uji])
print(prediksi)
print(label_uji.iloc[indeks_uji])

Setosa
Setosa


Lakukan pengujian untuk semua data uji. Selanjutnya bandingkan dengan label sebenarnya untuk menghitung berapa banyak data uji yang berhasil diprediksi dengan benar

In [23]:
prediksi_total = []
for indeks in range(data_uji.shape[0]) :
  prediksi_total.append(testing_naive_bayes_gaussian(model, data_uji.iloc[indeks]))
print("Total prediksi benar: ", sum(prediksi_total==label_uji))

Total prediksi benar:  30


## TUGAS
Pada tugas kali ini Anda akan menggunakan dataset **[Car Evaluation Dataset](https://archive.ics.uci.edu/ml/datasets/Car+Evaluation)** yang telah dimodifikasi. Fitur-fitur yang ada bertipe kategori sebagai berikut:
1. **buying**. Merupakan harga beli mobil. Nilai : vhigh, high, med, low
2. **maint**. Menandakan biaya perawatan mobil. Nilai: vhigh, high, med, low
3. **lug_boot**. Menandakan ukuran bagasi. Nilai: small, med, big
4. **safety**. Menandakan skor keamanan mobil. Nilai:  low, med, high
5. **class**. Merupakan kelas data. Nilai: unacc, acc, good, vgood

Download dataset dari https://dataset-ppm.s3.amazonaws.com/car_sample.csv dan letakkan dalam variabel **data_tugas**.

In [51]:
! wget --no-check-certificate "https://drive.google.com/uc?export=download&id=1RXcXrwkVoOI-hPvfJM1-FlmO_bB7M3YJ&quot" -O car_evaluation.csv

--2024-03-28 04:41:41--  https://drive.google.com/uc?export=download&id=1RXcXrwkVoOI-hPvfJM1-FlmO_bB7M3YJ&quot
Resolving drive.google.com (drive.google.com)... 142.251.170.113, 142.251.170.102, 142.251.170.101, ...
Connecting to drive.google.com (drive.google.com)|142.251.170.113|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1RXcXrwkVoOI-hPvfJM1-FlmO_bB7M3YJ&export=download [following]
--2024-03-28 04:41:42--  https://drive.usercontent.google.com/download?id=1RXcXrwkVoOI-hPvfJM1-FlmO_bB7M3YJ&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 108.177.125.132, 2404:6800:4008:c01::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|108.177.125.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 43263 (42K) [application/octet-stream]
Saving to: ‘car_evaluation.csv’

car_evaluation.csv  100%[===================>]  42.2

In [ ]:
!wget https://dataset-ppm.s3.amazonaws.com/car_sample.csv

--2020-11-04 01:24:09--  https://dataset-ppm.s3.amazonaws.com/car_sample.csv
Resolving dataset-ppm.s3.amazonaws.com (dataset-ppm.s3.amazonaws.com)... 52.217.66.108
Connecting to dataset-ppm.s3.amazonaws.com (dataset-ppm.s3.amazonaws.com)|52.217.66.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7226 (7.1K) [text/csv]
Saving to: ‘car_sample.csv’

car_sample.csv      100%[===================>]   7.06K  --.-KB/s    in 0s      

2020-11-04 01:24:10 (190 MB/s) - ‘car_sample.csv’ saved [7226/7226]



In [52]:
import pandas as pd
data_tugas = pd.read_csv('car_evaluation.csv')
data_tugas.head()

,buying,maint,lug_boot,safety,class
0,vhigh,vhigh,small,low,unacc
1,vhigh,vhigh,small,med,unacc
2,vhigh,vhigh,small,high,unacc
3,vhigh,vhigh,med,low,unacc
4,vhigh,vhigh,med,med,unacc


Bagi data menjadi data latih dan uji dengan nama **tugas_latih** dan **tugas_uji** dengan rasio 70%:30%

In [53]:
from sklearn.model_selection import train_test_split
tugas_latih, tugas_uji = train_test_split(data_tugas, test_size = 0.3)

Pisahkan label/kategori dari data uji menjadi variabel tersendiri. Beri nama variabelnya **tugas_label_uji**

In [54]:
tugas_label_uji = tugas_uji.pop('class')
tugas_uji.head()

,buying,maint,lug_boot,safety
353,vhigh,low,small,high
1,vhigh,vhigh,small,med
1584,low,med,small,low
655,high,med,big,med
537,high,vhigh,big,low


Hitunglah nilai prior dari data latih dan simpan nilainya pada variabel bernama **prior_tugas**. Anda dapat menggunakan fungsi **hitung_prior** yang telah Anda buat.

In [55]:
from collections import Counter
def hitung_prior(list_kelas) :
  n_data = len(list_kelas)
  prior_tugas = Counter(list_kelas)
  for key in prior_tugas.keys() :
    prior_tugas[key]=prior_tugas[key]/n_data
  return prior_tugas

prior_tugas = hitung_prior(tugas_latih['class'])
print(prior_tugas)

Counter({'unacc': 0.6931348221670802, 'acc': 0.22497932175351532, 'good': 0.04218362282878412, 'vgood': 0.03970223325062035})


Buatlah fungsi bernama **hitung_likelihood_multinomial** dengan parameter input berupa data latih. Fungsi ini bertujuan menghitung nilai likelihood pada masing-masing kolom pada data latih. Operasi-operasi yang perlu dilakukan pada fungsi ini adalah:


1.   Mendapatkan daftar kolom data
2.   Mendapatkan nama kolom kelas pada data (kolom terakhir pada data)
3.   Mendapatkan nama-nama kelas pada data (unik).
4.   Menghitung likelihood pada masing-masing kolom, nilai kolom, dan nilai kelas

Sebagai saran, Anda dapat menggunakan struktur data dictionary untuk menyimpan nilai likelihood. Key pada dictionary tersebut adalah tuple(nama kolom, nilai pada kolom, nama kelas). Petunjuk penggunaan dictionary dengan key berupa tuple dapat dibaca [di sini](https://stackoverflow.com/questions/1938614/in-what-case-would-i-use-a-tuple-as-a-dictionary-key).



In [56]:
def hitung_likelihood_multinomial(data_latih):
  #tulis kode Anda disini
  list_columns = data_latih.columns[:-1]
  class_column_name = data_latih.columns[-1]
  list_class = set(data_latih[class_column_name])
  list_value = ["low","med","high","vgood"]
  likelihood = {}
  for column in list_columns:
    for a_class in list_class:
      for value in list_value:
        cwc = np.sum(data_latih.loc[data_latih[class_column_name]==a_class][column]==value)
        cc = len(data_latih.loc[data_latih[class_column_name]==a_class][column]==value)
        likelihood[(column,value,a_class)] = cwc/cc
  return likelihood

Ujilah fungsi **hitung_likelihood_multinomial** dengan parameter input berupa variabel **tugas_latih**

In [57]:
likelihood = hitung_likelihood_multinomial(tugas_latih)
print(likelihood)

{('buying', 'low', 'unacc'): 0.21837708830548927, ('buying', 'med', 'unacc'): 0.23031026252983294, ('buying', 'high', 'unacc'): 0.2613365155131265, ('buying', 'vgood', 'unacc'): 0.0, ('buying', 'low', 'good'): 0.6862745098039216, ('buying', 'med', 'good'): 0.3137254901960784, ('buying', 'high', 'good'): 0.0, ('buying', 'vgood', 'good'): 0.0, ('buying', 'low', 'acc'): 0.23161764705882354, ('buying', 'med', 'acc'): 0.3272058823529412, ('buying', 'high', 'acc'): 0.27205882352941174, ('buying', 'vgood', 'acc'): 0.0, ('buying', 'low', 'vgood'): 0.5625, ('buying', 'med', 'vgood'): 0.4375, ('buying', 'high', 'vgood'): 0.0, ('buying', 'vgood', 'vgood'): 0.0, ('maint', 'low', 'unacc'): 0.21957040572792363, ('maint', 'med', 'unacc'): 0.2159904534606205, ('maint', 'high', 'unacc'): 0.2649164677804296, ('maint', 'vgood', 'unacc'): 0.0, ('maint', 'low', 'good'): 0.6274509803921569, ('maint', 'med', 'good'): 0.37254901960784315, ('maint', 'high', 'good'): 0.0, ('maint', 'vgood', 'good'): 0.0, ('main

Buatlah fungsi bernama **training_naive_bayes_multinomial** dengan input berupa data latih. Fungsi ini bertujuan membentuk model yang berisi:


1.   Nilai prior semua kelas
2.   Nilai likelihood semua kolom, nilai kolom, dan kelas
3.   Daftar kelas pada data latih (unik)
4.   Daftar nama kolom pada data latih

Return value dari fungsi ini adalah **model**



In [58]:
def training_naive_bayes_multinomial(data_latih):
  #tulis kode Anda disini
  class_column_name = data_latih.columns[-1]
  prior = hitung_prior(data_latih[class_column_name])
  likelihood = hitung_likelihood_multinomial(data_latih)
  list_class = set(data_latih[class_column_name])
  list_columns = data_latih.columns[:-1]
  model = {}
  model['prior'] = prior
  model['likelihood'] = likelihood
  model['list_class'] = list_class
  model['list_columns'] = list_columns
  return model

Jalankan fungsi **training_naive_bayes_multinomial** dan simpan hasilnya dalam variabel **model_tugas**

In [59]:
model_tugas = training_naive_bayes_multinomial(tugas_latih)

Buatlah fungsi **testing_naive_bayes_multinomial** untuk mendapatkan kelas dari sebuah data uji. Langkah yang dilakukan hampir sama dengan Gaussian, yaitu mengalikan likelihood semua kolom dengan posterior.

In [60]:
def testing_naive_bayes_multinomial(model,data_uji):
  #tulis kode Anda disini
  prior = model['prior']
  likelihood = model['likelihood']
  list_class = model['list_class']
  list_columns = model['list_columns']
  posterior = dict.fromkeys(list_class,1)
  for a_class in list_class:
    for column in list_columns:
      for a_likelihood in likelihood:
        posterior[a_class]   = posterior[a_class]*likelihood[a_likelihood]
    posterior[a_class] = posterior[a_class]*prior[a_class]
  kelas_uji = max(posterior, key = posterior.get)
  return kelas_uji

Lakukan pengujian menggunakan sebuah data uji

In [61]:
idx = 10
uji = tugas_uji.iloc[idx]
kelas_uji = testing_naive_bayes_multinomial(model_tugas,uji)
print(kelas_uji)
print(tugas_label_uji.iloc[idx])

unacc
acc


Lakukan pengujian terhadap semua data uji.

In [62]:
prediksi_total =[]
for indeks in range(tugas_uji.shape[0]):
  prediksi_total.append(testing_naive_bayes_multinomial(model_tugas,tugas_uji.iloc[indeks]))
print("Total prediksi benar: ",sum(prediksi_total==tugas_label_uji))

Total prediksi benar:  372


In [63]:
len(tugas_label_uji)

519